In [1]:
import numpy as np
from scipy.signal import chirp
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
import os
import h5py

In [2]:
# Reads plt files, converts into numpy array

def velfield(ts, boa, length):
    
    path = '/Volumes/KleeFD/ibpm raw data/klee_chirp_BoA'+str(boa)+'_f0p05f0p5_L0'+str(length)+'/ibpm'\
                + str(format(ts, '05'))+'.plt'
    with open(path) as file:
        f_text = file.read()

    f_list = f_text[191:]
    f_list = f_list.split(' ')
    f_list.pop()

    UX, UY = [], []
    for idx, val in enumerate(f_list):
        if val[0] == '\n':
            UX.append(float(f_list[idx+2]))
            UY.append(float(f_list[idx+3]))
    UX = np.array(UX); UY = np.array(UY)

    UX = UX.reshape(299, 599)
    UY = UY.reshape(299, 599)

    return UX, UY

In [3]:
# This is how timestep, freq, angle is defined

ibpm_time = np.arange(5510, 11500, 10)
freq = 0.05 + ((ibpm_time - 5501)/100)/60*(0.5-0.05)

ori_time = np.arange(0, 60, 0.01)

angle = 5*chirp(ori_time, 0.05, 30, 0.5, method = 'linear', phi = 0)
angle = angle[9::10][:-1]

In [4]:
# Categorize the continuous frequency

freq_cat = np.array([0, 0.05, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6])

# Set threshold values to be the mean value between two categoires. i.e. if 0.075 < x < 0.15 then 0.1
thres = []
for i in range(8):
    thres.append(tuple([(freq_cat[i+1] + freq_cat[i])/2, (freq_cat[i+1] + freq_cat[i+2])/2]))

freq_cat = np.array([0.05, 0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5])

categorized_freq = []
for i, f in enumerate(freq):
    for cat_num in range(8):
        bound = thres[cat_num]
        if bound[0] < f and bound[1] > f:
            categorized_freq.append(freq_cat[cat_num])
            
categorized_freq = np.array(categorized_freq)

# Chirp BoA 25

In [5]:
# Load Chirp Data for boa 25

chirp_X25 = np.zeros((len(ibpm_time), 200, 200))

for i, timestep in enumerate(ibpm_time):
    
    UX, UY = velfield(timestep, 25, 6000)
    
    UX = UX[99:199, 399:599]
    UY = UY[99:199, 399:599]
    
    chirp_X25[i] = np.concatenate((UX, UY))
    
chirp_X25 = chirp_X25.reshape(len(ibpm_time), 200, 200,1)

boa = (np.full((len(ibpm_time), 1), 25.0)).reshape(-1,1)
typ = (np.full((len(ibpm_time), 1), 'chirp')).reshape(-1,1)
inst_angle = (25 - angle).reshape(-1,1)
inst_freq = freq.reshape(-1,1)

Y = np.column_stack((boa, categorized_freq, typ, inst_angle, inst_freq))


# Split Chirp dataset into train : test : validation = 70 : 15 : 15
train_X25, val_X25, train_Y25, val_Y25 = \
    train_test_split(chirp_X25, Y, test_size = 0.3, random_state = 0, shuffle = True, stratify = Y[:][:,0:2])

test_X25, val_X25, test_Y25, val_Y25 = \
    train_test_split(val_X25, val_Y25, test_size = 0.5, random_state = 0, shuffle = True, stratify = val_Y25[:][:,0:2])

# Check stratification
print("training :", Counter(train_Y25[:,0]), Counter(train_Y25[:, 1]))
print('validation :', Counter(val_Y25[:,0]), Counter(val_Y25[:, 1]))
print('test :', Counter(test_Y25[:,0]), Counter(test_Y25[:, 1]))

training : Counter({'25.0': 419}) Counter({'0.1': 70, '0.4': 70, '0.2': 70, '0.25': 47, '0.35': 47, '0.3': 46, '0.5': 46, '0.05': 23})
validation : Counter({'25.0': 90}) Counter({'0.4': 15, '0.1': 15, '0.2': 15, '0.35': 10, '0.5': 10, '0.3': 10, '0.25': 10, '0.05': 5})
test : Counter({'25.0': 90}) Counter({'0.4': 15, '0.1': 15, '0.2': 15, '0.3': 10, '0.5': 10, '0.35': 10, '0.25': 10, '0.05': 5})


# Chirp BoA 30

In [8]:
# Load Chirp Data for boa 30

chirp_X30 = np.zeros((len(ibpm_time), 200, 200))

for i, timestep in enumerate(ibpm_time):
    
    UX, UY = velfield(timestep, 30, 6000)
    
    UX = UX[99:199, 399:599]
    UY = UY[99:199, 399:599]
    
    chirp_X30[i] = np.concatenate((UX, UY))
    
chirp_X30 = chirp_X30.reshape(len(ibpm_time), 200, 200,1)

boa = (np.full((len(ibpm_time), 1), 30.0)).reshape(-1,1)
typ = (np.full((len(ibpm_time), 1), 'chirp')).reshape(-1,1)
inst_angle = (30 - angle).reshape(-1,1)

Y = np.column_stack((boa, categorized_freq, typ, inst_angle, inst_freq))


# Split Chirp dataset into train : test : validation = 70 : 15 : 15
train_X30, val_X30, train_Y30, val_Y30 = \
    train_test_split(chirp_X30, Y, test_size = 0.3, random_state = 0, shuffle = True, stratify = Y[:][:,0:2])

test_X30, val_X30, test_Y30, val_Y30 = \
    train_test_split(val_X30, val_Y30, test_size = 0.5, random_state = 0, shuffle = True, stratify = val_Y30[:][:,0:2])


# Check stratification
print("training :", Counter(train_Y30[:,0]), Counter(train_Y30[:, 1]))
print('validation :', Counter(val_Y30[:,0]), Counter(val_Y30[:, 1]))
print('test :', Counter(test_Y30[:,0]), Counter(test_Y30[:, 1]))

training : Counter({'30.0': 419}) Counter({'0.1': 70, '0.4': 70, '0.2': 70, '0.25': 47, '0.35': 47, '0.3': 46, '0.5': 46, '0.05': 23})
validation : Counter({'30.0': 90}) Counter({'0.4': 15, '0.1': 15, '0.2': 15, '0.35': 10, '0.5': 10, '0.3': 10, '0.25': 10, '0.05': 5})
test : Counter({'30.0': 90}) Counter({'0.4': 15, '0.1': 15, '0.2': 15, '0.3': 10, '0.5': 10, '0.35': 10, '0.25': 10, '0.05': 5})


In [9]:
train_X = np.concatenate((train_X25, train_X30), axis = 0)
val_X = np.concatenate((val_X25, val_X30), axis = 0)
test_X = np.concatenate((test_X25, test_X30), axis = 0)

train_Y = np.concatenate((train_Y25, train_Y30), axis = 0)
val_Y = np.concatenate((val_Y25, val_Y30), axis = 0)
test_Y = np.concatenate((test_Y25, test_Y30), axis = 0)

In [10]:
d = '/Volumes/KleeFD/dataset/dataset_chirp/05_faster chirp/'

np.save(d+'train_X.npy', train_X)
np.save(d+'train_Y.npy', train_Y)

np.save(d+'val_X.npy', val_X)
np.save(d+'val_Y.npy', val_Y)

np.save(d+'test_X.npy', test_X)
np.save(d+'test_Y.npy', test_Y)